## Homework 2 - Shany Herskovits (040379604) and Ofir Nahshon (038683363)


# Consumers Notebook

## First run this notebook and then the Producers notebook and return to see the online results

In [1]:
from confluent_kafka import Consumer

conf = {'bootstrap.servers': "localhost:9092",
        'group.id': "foo",
        'auto.offset.reset': 'smallest'}

consumer = Consumer(conf)

In [2]:

# Shared RDDs
sentiment_rdd = sc.parallelize([])  # Initially empty
text_rdd = sc.parallelize([])       # Initially empty

def process_sentiments(sentiments_data):
    """Update the sentiment dictionary RDD with incoming data."""
    global sentiment_rdd

    # Convert incoming data into RDD format
    new_sentiments_rdd = sc.parallelize(sentiments_data)  # [(word, score), ...]
    print(f"Processing {len(sentiments_data)} sentiment updates.")
    # Update sentiment RDD: merge with new data and keep the latest score
    sentiment_rdd = sentiment_rdd.union(new_sentiments_rdd).reduceByKey(lambda a, b: b)
    print(f'Total sentiment table length is {sentiment_rdd.count()}')
    
def analyze_text(text_data):
    """Perform sentiment analysis on incoming text data."""
    global sentiment_rdd

    # Convert incoming data into RDD format
    new_text_rdd = sc.parallelize(text_data)  # ["sentence1", "sentence2", ...]

    # Broadcast the sentiment dictionary
    sentiment_dict = sc.broadcast(dict(sentiment_rdd.collect()))

    # Perform sentiment analysis
    result_rdd = new_text_rdd.flatMap(lambda line: line.split()) \
                             .map(lambda word: sentiment_dict.value.get(word, 0)) \
                             .filter(lambda score: score != 0) \
                             .map(lambda score: ("Total Sentiment Level", score)) \
                             .reduceByKey(lambda a, b: a + b)

    # Collect and print results
    results = result_rdd.collect()
    for key, value in results:
        print(f"{key}: {value} for {len(text_data)} text batches anaylized")
        
    


In [3]:
def consume_loop(consumer, topics):
    try:
        consumer.subscribe(topics)
        while True:
            messages = consumer.consume(num_messages=100,timeout=1.0)  # Batch size of 100
            if not messages:
                continue

            # Group messages by topic
            sentiments_batch = []
            text_batch = []

            for msg in messages:
                if msg.error():
                    if msg.error().code() == KafkaError._PARTITION_EOF:
                        # End of partition event
                        continue
                    else:
                        raise KafkaException(msg.error())
                else:
                    if msg.topic() == 'sentiments':
                        # Parse message into (word, score)
                        word, score = msg.value().decode('utf-8').split(":")
                        sentiments_batch.append((word, int(score)))
                    elif msg.topic() == 'text':
                        # Add full message to text batch
                        text_batch.append(msg.value().decode('utf-8'))

            # Process batches
            if sentiments_batch:
                process_sentiments(sentiments_batch)
            
            if text_batch:
                analyze_text(text_batch)
    finally:
        # Close down consumer to commit final offsets.
        consumer.close()


In [ ]:
# Start the consumer loop
consume_loop(consumer, ['sentiments', 'text'])

Processing 99 sentiment updates.


Total sentiment table length is 99
Processing 99 sentiment updates.


Total sentiment table length is 198
Total Sentiment Level: 1 for 1 text batches anaylized
Processing 99 sentiment updates.


Total sentiment table length is 297
Processing 99 sentiment updates.


Total sentiment table length is 396


...

Total Sentiment Level: 13 for 1 text batches anaylized


Total Sentiment Level: 14 for 2 text batches anaylized


Total Sentiment Level: 1 for 1 text batches anaylized


Total Sentiment Level: 16 for 2 text batches anaylized


Total Sentiment Level: 1 for 1 text batches anaylized
